In [3]:
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, GlobalAveragePooling2D, Input, Concatenate, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.applications import MobileNetV2, EfficientNetB3
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras import regularizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam

In [4]:
import os
path = os.listdir('C:/SEMESTER 5/STECHOQ/PROJECT/data/brain/Training/')
classes = {'glioma_tumor':0, 'meningioma_tumor':1, 'no_tumor':2, 'pituitary_tumor':3}

### GET DATA

In [5]:
import cv2
X = []
Y = []
for cls in classes:
    pth = 'C:/SEMESTER 5/STECHOQ/PROJECT/data/brain/Training/'+cls
    for j in os.listdir(pth):
        img = cv2.imread(pth+'/'+j, 0)
        img = cv2.resize(img, (200,200))
        X.append(img)
        Y.append(classes[cls])

In [6]:
X = np.array(X)
Y = np.array(Y)

X_updated = X.reshape(len(X), -1)

In [7]:
np.unique(Y)

array([0, 1, 2, 3])

In [8]:
pd.Series(Y).value_counts()

3    827
0    826
1    822
2    395
Name: count, dtype: int64

### SPLIT DATA

In [9]:
xtrain, xtest, ytrain, ytest = train_test_split(X_updated, Y, random_state=10, test_size=.20)

In [10]:
xtrain.shape, xtest.shape

((2296, 40000), (574, 40000))

### FEATURE SELECTION : PCA

In [11]:
print(xtrain.shape, xtest.shape)

pca = PCA(.98)
pca_train = pca.fit_transform(xtrain)
pca_test = pca.transform(xtest)
# pca_train = xtrain
# pca_test = xtest

(2296, 40000) (574, 40000)


In [12]:
def build_combined_model():
    input_shape = (224, 224, 3) 
    input_layer = Input(shape=input_shape)
    efficientnet_model = EfficientNetB3(include_top=False, input_tensor=input_layer, weights='imagenet')

    mobilenet_model = MobileNetV2(include_top=False, input_tensor=input_layer, weights='imagenet')

    efficientnet_layers = efficientnet_model.get_layer('block6a_expand_conv').output
    mobilenet_layers = mobilenet_model.get_layer('block_12_expand').output

    merged_layers = Concatenate()([efficientnet_layers, mobilenet_layers])

    combined_model = Model(inputs=input_layer, outputs=merged_layers)

    return combined_model

In [13]:
combined_model = build_combined_model()

combined_model.summary()



Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 rescaling (Rescaling)       (None, 224, 224, 3)          0         ['input_1[0][0]']             
                                                                                                  
 normalization (Normalizati  (None, 224, 224, 3)          7         ['rescaling[0][0]']           
 on)                                                                                              
                                                                                                  
 rescaling_1 (Rescaling)     (None, 224, 224, 3)          0         ['normalization[0][0]'] 

In [14]:
# Menambahkan kode untuk meng-compile model
combined_model.compile(optimizer=Adam(learning_rate=0.001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [15]:
num_epochs = 20
batch_size = 32
history = combined_model.fit(pca_train, ytrain, 
                            validation_data=(pca_test, ytest),
                            epochs=num_epochs, 
                            batch_size=batch_size)

Epoch 1/20


ValueError: in user code:

    File "c:\Users\A S U S\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\src\engine\training.py", line 1401, in train_function  *
        return step_function(self, iterator)
    File "c:\Users\A S U S\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\src\engine\training.py", line 1384, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "c:\Users\A S U S\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\src\engine\training.py", line 1373, in run_step  **
        outputs = model.train_step(data)
    File "c:\Users\A S U S\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\src\engine\training.py", line 1150, in train_step
        y_pred = self(x, training=True)
    File "c:\Users\A S U S\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\src\utils\traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from Non

In [16]:
train_path = os.path.join(output_path, 'train')
val_path = os.path.join(output_path, 'val')
test_path = os.path.join(output_path, 'test')

os.makedirs(train_path, exist_ok=True)
os.makedirs(val_path, exist_ok=True)
os.makedirs(test_path, exist_ok=True)

NameError: name 'output_path' is not defined

In [ ]:
# # Define input layers
# input_mobile = Input(shape=(img_width, img_height, 3), name="input_mobile")
# input_efficient = Input(shape=(img_width, img_height, 3), name="input_efficient")

In [ ]:
# # Load MobileNet model
# mobile_net_model = MobileNet(
#     include_top=False,
#     weights="imagenet",
#     input_shape=(img_width, img_height, 3),
#     pooling='max',
#     classifier_activation="softmax",
# )

In [ ]:
# # Load EfficientNetB3 model
# efficient_net_model = tf.keras.applications.EfficientNetB3(
#     include_top=False,
#     weights="imagenet",
#     input_shape=(img_width, img_height, 3),
#     pooling='max',
#     classifier_activation="softmax",
# )

In [ ]:
# # Set layers of both models as non-trainable
# for layer in mobile_net_model.layers:
#     layer.trainable = False

# for layer in efficient_net_model.layers:
#     layer.trainable = False

In [ ]:
# # Connect input layers to the models
# output_mobile = mobile_net_model(input_mobile)
# output_efficient = efficient_net_model(input_efficient)

In [ ]:
# # Concatenate the output of MobileNet and EfficientNetB3
# combined_output = Concatenate()([output_mobile, output_efficient])